<a href="https://colab.research.google.com/github/kangmyoungseok/Scam_Token_Feature/blob/main/Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pandas.core.frame import DataFrame
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import re 
import time
from multiprocessing import Pool

file_name = './Pairs_v1.2.csv'
datas = pd.read_csv(file_name).to_dict('records')
datas_len = len(datas)

def get_creatorAddress(data):
    token_id = data['token00.id']
    repos_url = 'https://api.ethplorer.io/getAddressInfo/'+token_id+'?apiKey=EK-4L18F-Y2jC1b7-9qC3N'
    response = requests.get(repos_url).text
    repos = json.loads(response)    #json 형태로 token_id에 해당하는 정보를 불러온다.
    
    try:
        creator_address = repos['contractInfo']['creatorAddress']
    except:     #오류가 나면 이더스캔에서 크롤링
        url = 'https://etherscan.io/address/'+token_id
        try:
            response = requests.get(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})
            page_soup = BeautifulSoup(response.text, "html.parser")
            Transfers_info_table_1 = str(page_soup.find("a", {"class": "hash-tag text-truncate"}))
            creator_address = re.sub('<.+?>', '', Transfers_info_table_1, 0).strip()
            print(token_id +'///'+creator_address)
        except Exception as e:  #이더스캔 크롤링까지 에러나면 'Error'로 표시
            creator_address = 'Error'
            print('token_id' + token_id)
            print(e)
        
    data['token00_creator_address'] = creator_address
    return data

if __name__=='__main__':
    p = Pool(8)
    count = 0
    result = []
    for ret in p.imap(get_creatorAddress,datas):
      count = count+1
      result.append(ret)
      if(count % 200 == 0):
        print("Process Rate : {}/{} {}%".format(count,datas_len,int((count/datas_len)*100)))
    p.close()
    p.join()
    df = pd.DataFrame(result)
    df.to_csv("V1.2+creatoraddress.csv",index=False)
 

In [13]:
#Etherplorer API / Etherscan Crawling으로 Creator Address를 구한다.
from pandas.core.frame import DataFrame
from requests import Request, Session
import pandas as pd
import json
from bs4 import BeautifulSoup
import re # 추가
from urllib.request import urlopen
import requests
import time
from multiprocessing import Pool

def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://graphql.bitquery.io/'
                            '',headers={'X-API-KEY': 'BQYgQRzGYhzys0AOlpdpipougQJMH1J8'},
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        print ('Query failed. return code is {}.      {}'.format(request.status_code, query))


def get_initial_ETH(data):
    pair_id= data['id']
    creator_Address = data['token00_creator_address'] 
    query = '{ ethereum { transfers(options: {asc: "block.timestamp.time", limit: 1} amount: {gt: 0} receiver: {is: "%s"} sender: {is: "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"} txFrom: {is : "%s"} ) { block {timestamp {time}} currency { symbol } amount }}}' % (pair_id,creator_Address)
    result = run_query(query)
    try:
        ETH_amount = result['data']['ethereum']['transfers'][0]['amount']
    except Exception as e:
        ETH_amount = 'Error'
        print(e)
    data['initial_Liquidity_ETH'] = ETH_amount
    return data

def split_csv(total_csv):
    rows = pd.read_csv(total_csv,chunksize=5000)
    file_count = 0
    for i, chuck in enumerate(rows):
        chuck.to_csv('out{}.csv'.format(i))
        file_count = file_count+1 
    return file_count

def get_creatorAddress(data):
    token_id = data['token00.id']
    repos_url = 'https://api.ethplorer.io/getAddressInfo/'+token_id+'?apiKey=EK-4L18F-Y2jC1b7-9qC3N'
    response = requests.get(repos_url).text
    repos = json.loads(response)    #json 형태로 token_id에 해당하는 정보를 불러온다.
    
    try:
        creator_address = repos['contractInfo']['creatorAddress']
    except:     #오류가 나면 이더스캔에서 크롤링
        url = 'https://etherscan.io/address/'+token_id
        try:
            response = requests.get(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})
            page_soup = BeautifulSoup(response.text, "html.parser")
            Transfers_info_table_1 = str(page_soup.find("a", {"class": "hash-tag text-truncate"}))
            creator_address = re.sub('<.+?>', '', Transfers_info_table_1, 0).strip()
        except Exception as e:  #이더스캔 크롤링까지 에러나면 'Error'로 표시
            creator_address = 'Error'
            print(e)

    data['token00_creator_address'] = creator_address
    return data

if __name__=='__main__':
    file_name = './Pairs_v1.2.csv'
    file_count = split_csv(file_name)

    for i in range(file_count):         #하나의 파일 단위로 Creator Address 불러오고, 해당 초기 유동성풀 이더값 구해온다.
        file_name = 'out{}.csv'.format(i)
        datas = pd.read_csv(file_name).to_dict('records')
        datas_len = len(datas)
        try:
            p = Pool(8)
            count = 0
            result = []
            for ret in p.imap(get_creatorAddress,datas):
                count = count+1
                result.append(ret)
                if(count % 200 == 0):
                    print("Process Rate : {}/{} {}%".format(count,datas_len,int((count/datas_len)*100)))
            p.close()
            p.join()
        except Exception as e:
            print(e)
        print(result)
        print('=======')
        time.sleep(5)
        p2 = Pool(8)
        count = 0
        result2 = []
        for ret in p2.imap(get_initial_ETH,result):
            count = count+1
            result2.append(ret)
            if(count % 200 == 0):
                print("Process Rate : {}/{} {}%".format(count,datas_len,int((count/datas_len)*100)))
        p2.close()
        p2.join()
            
        df = pd.DataFrame(result2)
        file_name = 'fout{}.csv'.format(i)
        df.to_csv(file_name)
    
    

Process Rate : 200/5000 4%


Process ForkPoolWorker-121:
Process ForkPoolWorker-123:
Process ForkPoolWorker-124:
Traceback (most recent call last):


KeyboardInterrupt: ignored

Process ForkPoolWorker-122:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
